<a href="https://colab.research.google.com/github/altaga/TwitterClusterMexico/blob/master/Trabajadores_Tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
import pandas
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [0]:
print(tf.__version__)

2.0.0


In [0]:
# Los datos
url  = 'https://raw.githubusercontent.com/altaga/AI-Work-Hey-I-work-/master/Employee%20Data.csv?token=AD6LORXLXBRG3LMGYDCRXDK53OA74'

In [0]:
datos = pandas.read_csv(url)
datos

,Age,Distance,Gender,Years,Grade,Time
0,20,23,1,0,2,2
1,20,3,0,0,2,19
2,18,6,0,0,2,22
3,19,6,1,0,1,22
4,19,5,1,0,2,17
...,...,...,...,...,...,...
95,47,23,1,15,8,-21
96,41,5,0,16,9,-60
97,47,13,1,16,8,-14
98,45,20,1,16,6,-33


In [0]:
# Las 15 primeras columnas son los características
caracteristicas= datos.values[:,:5]
# La última columna se coonvierte en los etiquetas
etiquetas= datos.values[:,5]
etiquetas

array([  2,  19,  22,  22,  17,   3,  16,  28,  26,  21,   7,   2,   3,
        12,  11,   8,  11,  15,  13,   3,  10,  15,   7,   9,   7,   2,
        -2,   1,   1,   6,   9,   7,   0,   4,  -5,  -1,   3,   2,   5,
         5,  -2,   9,   2,  -3,   2,   5,   6,  -3,   1,   1,  -9, -11,
        -6,  -8, -11, -15,  -5, -12,  -9,  -6, -11,  -3,  -5, -10, -14,
       -12,  -5, -12, -10, -12,  -6, -11, -28, -29, -18, -20, -23, -21,
       -16,  -1, -15, -10,  -2, -28,  -1, -21, -14, -50, -18, -22, -23,
       -45, -35, -30, -31, -21, -60, -14, -33, -48])

In [0]:
# Normalizamos las carasterísticas. Las dividimos entre 255
caracteristicas_norm= caracteristicas/255

In [0]:
# Dividimos los datos en estrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(caracteristicas_norm, etiquetas, test_size=0.2, random_state=21)

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation=tf.nn.relu, input_shape=[5]),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
  ])
optimizer = tf.keras.optimizers.RMSprop(0.0099)
model.compile(loss='mean_squared_error',optimizer=optimizer)

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                192       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 2,337
Trainable params: 2,337
Non-trainable params: 0
_________________________________________________________________


In [0]:
# El entrenamiento
model.fit(X_train,y_train,epochs=500)

Train on 80 samples
Epoch 1/500
80/80 [==============================] - 2s 21ms/sample - loss: 332.0512
Epoch 2/500
80/80 [==============================] - 0s 167us/sample - loss: 295.5939
Epoch 3/500
80/80 [==============================] - 0s 162us/sample - loss: 270.6455
Epoch 4/500
80/80 [==============================] - 0s 131us/sample - loss: 259.3866
Epoch 5/500
80/80 [==============================] - 0s 141us/sample - loss: 254.5939
Epoch 6/500
80/80 [==============================] - 0s 145us/sample - loss: 235.0627
Epoch 7/500
80/80 [==============================] - 0s 142us/sample - loss: 230.0541
Epoch 8/500
80/80 [==============================] - 0s 143us/sample - loss: 210.3585
Epoch 9/500
80/80 [==============================] - 0s 135us/sample - loss: 180.0280
Epoch 10/500
80/80 [==============================] - 0s 134us/sample - loss: 170.1503
Epoch 11/500
80/80 [==============================] - 0s 144us/sample - loss: 139.1870
Epoch 12/500
80/80 [=============

In [0]:
# La evaluación
score = model.evaluate(X_test, y_test)
test = [45,12,0,17,9]
x= np.array(test)
x= np.array([x])/255
x = [[0.94, 0.08, 0.02, 0.24, 0.1 ]]
res = model.predict(x)
print(res)

20/1 [========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 172us/sample - loss: 22.3168
[[-26.692406]]


In [0]:
# Stay here
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

# Save the model to disk
open("exercice_model.tflite", "wb").write(tflite_model)

5080

In [0]:
X_test = X_test.astype('float32')

# Instantiate an interpreter for each model
exercice_model = tf.lite.Interpreter('exercice_model.tflite')

# Allocate memory for each model
exercice_model.allocate_tensors()

# Get the input and output tensors so we can feed in values and get the results
input_details = exercice_model.get_input_details()
output_details = exercice_model.get_output_details()

print(input_details)

test = 8

input_data = X_test[test].reshape((1,5))

print(input_data)

exercice_model.set_tensor(input_details[0]['index'],input_data )

exercice_model.invoke()
  
output_data = exercice_model.get_tensor(output_details[0]['index'])
    
  
print(output_data)  
print(y_test[test])    

[{'name': 'dense_4_input', 'index': 1, 'shape': array([1, 5], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[[0.94 0.08 0.02 0.24 0.1 ]]
[[-26.545229]]
-30


In [0]:
# aquí termina el 2do tutorial

In [0]:
import os
basic_model_size = os.path.getsize("exercice_model.tflite")
print("Basic model is %d bytes" % basic_model_size)

Basic model is 5080 bytes


In [0]:
# Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i exercice_model.tflite > exercice_model.cc
# Print the source file
!cat exercice_model.cc

Selecting previously unselected package xxd.
(Reading database ... 145605 files and directories currently installed.)
Preparing to unpack .../xxd_2%3a8.0.1453-1ubuntu1.1_amd64.deb ...
Unpacking xxd (2:8.0.1453-1ubuntu1.1) ...
Setting up xxd (2:8.0.1453-1ubuntu1.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
unsigned char exercice_model_tflite[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x12, 0x00, 0x1c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00,
  0x10, 0x00, 0x14, 0x00, 0x00, 0x00, 0x18, 0x00, 0x12, 0x00, 0x00, 0x00,
  0x03, 0x00, 0x00, 0x00, 0x90, 0x13, 0x00, 0x00, 0xa8, 0x0d, 0x00, 0x00,
  0x90, 0x0d, 0x00, 0x00, 0x34, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x18, 0xed, 0xff, 0xff,
  0x08, 0x00, 0x00, 0x00, 0x0e, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00,
  0x6d, 0x69, 0x6e, 0x5f, 0x72, 0x75, 0x6e, 0x74, 0x69, 0x6d, 0x65, 0x5f,
  0x76, 0x65, 0x72, 0x73, 0x69, 0x6f, 0x6e,